In [20]:
# web get, terminal command using!
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-02-02 02:50:22--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.09s   

2025-02-02 02:50:22 (11.8 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [21]:
with open("input.txt", "r", encoding="utf-8") as f:
  text = f.read()

print(f"Length of dataset in chars: {len(text)}")

Length of dataset in chars: 1115394


In [22]:
print(text[:11])

First Citiz


In [43]:
vocab = sorted(list(set(text)))
vocab_size = len(vocab)
print(vocab_size)
print("".join(vocab))
print("".join(list(set(vocab))))



65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
v&;igXFe:!I3kaJh'flpKP,- MZwtym?D.qnLQVUHGSWB
xu$zARdbTjrNcEYCoOs


In [75]:
str_to_int = {c:i for i, c in enumerate(vocab)}
int_to_str = {i:c for i, c in enumerate(vocab)}
encode = lambda x: [str_to_int[c] for c in x]
decode = lambda x: "".join([int_to_str[n] for n in x])

encoded = encode("Messi")
print(encoded)
decoded = decode(encoded)
print(decoded)


[25, 43, 57, 57, 47]
Messi


In [25]:
# encode the entire text dataset and store it in a torch.tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [26]:
# Lets now split up the data into train and validation sets

n = int(0.9 * len(data))
train_data = data[:n+1]
print(len(data[:n+1]) / len(data))
test_data = data[n+1:]
print(len(data[n+1:]) / len(data))

0.9000003586176724
0.09999964138232768


In [27]:
# transformers arent trained on data all at once can be expensive
# instead when we train we work with chunk of the dataset
# chunks at a time
# they have a length max_length called block_size or context_length

block_size = 8
train_data[:block_size+1]

# this is one example of context_size 8 but within it has 8 examples bc from first token will try to
# predict what token comes next
# gpt would predict what word comes next thats why the vocab of the model is 50k while ours is 65 because more unique
# words than unique letters, but then when we encode a token when using a small vocab the encoding is
# longer in length than the encoding length of gpt token bc would be able to cover a word with one number
# than our tokenization which is one word with manyy numbers

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [28]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for i in range(block_size):
  context = train_data[:i+1]
  target = y[i]
  print(f"When input is {context} the target is {target}")


When input is tensor([18]) the target is 47
When input is tensor([18, 47]) the target is 56
When input is tensor([18, 47, 56]) the target is 57
When input is tensor([18, 47, 56, 57]) the target is 58
When input is tensor([18, 47, 56, 57, 58]) the target is 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [29]:
# This is helpful for our transformer because it will be helpful in inference or testing
# the transformer will get used to predicting next token with context of one token or up to the
# context length (block_size). Transformer will never receive more context than context length / block size
# so when we are at limit we truncate the beginning of the context window and will have space to add another
# token to the front

In [30]:
# so far this is time dimension so we see how over time we need to see if the transformer can predict next token
# batch dimension is stacking up more than one example at same time bc of gpu parallelism
# multiple chunks of text


In [31]:
torch.manual_seed(1337) # to sample random locations in the data set to pull chunks from of length context_size this will ensure that the randint will be same for us
batch_size = 4 # how many sequences every forward backwards pass for transformer
block_size = 8

def get_batch(split):
  # generate a small batch of data inputs x and targets y
  data = train_data if split == "train" else test_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  # will get a 4 by 8 tensor
  # 4 sequences, each sequence size 8
  # the y at each corresponding will be the target the x needs
  # to predict next
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+1+block_size] for i in ix])
  return x,y

xb, yb = get_batch("train")
print("inputs:")
print(xb.shape)
print(xb)
print("targets")
print(yb.shape)
print(yb)

print("-----")

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"Batch #{b}, When input is {context}, the target is {target}")

# We covered time dimension now and the batch dimension


inputs:
torch.Size([4, 8])
tensor([[56,  6,  0, 24, 43, 58,  1, 61],
        [39, 47, 51,  1, 58, 46, 39, 58],
        [52, 45,  1, 58, 53,  1, 57, 39],
        [43, 47, 52, 45,  1, 46, 53, 50]])
targets
torch.Size([4, 8])
tensor([[ 6,  0, 24, 43, 58,  1, 61, 46],
        [47, 51,  1, 58, 46, 39, 58,  1],
        [45,  1, 58, 53,  1, 57, 39, 63],
        [47, 52, 45,  1, 46, 53, 50, 47]])
-----
Batch #0, When input is tensor([56]), the target is 6
Batch #0, When input is tensor([56,  6]), the target is 0
Batch #0, When input is tensor([56,  6,  0]), the target is 24
Batch #0, When input is tensor([56,  6,  0, 24]), the target is 43
Batch #0, When input is tensor([56,  6,  0, 24, 43]), the target is 58
Batch #0, When input is tensor([56,  6,  0, 24, 43, 58]), the target is 1
Batch #0, When input is tensor([56,  6,  0, 24, 43, 58,  1]), the target is 61
Batch #0, When input is tensor([56,  6,  0, 24, 43, 58,  1, 61]), the target is 46
Batch #1, When input is tensor([39]), the target is 4

In [32]:
# We have 4 batches, each have time dimension context length / block size
# of 8 so we really have 32 examples for the transformer to learn
# during training

In [77]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    logits = self.token_embedding_table(idx)
    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T, C)
    # targets = targets.view(-1) let pytorch lay it out
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for __ in range(max_new_tokens):
      # doing this will pass the xb into the forward method
      logits, loss = self(idx)

      # We want the logits for the last token only since generating
      # based on current context
      logits = logits[:, -1, :]

      # normalize using softmax 0 to 1 on dim 1 because the logits are in tensor
      # [[......]]
      probs = F.softmax(logits, dim = -1)

      # torch.multinomial samples one token per batch element
      # based on probs in probs
      idx_next = torch.multinomial(probs, num_samples=1)
      res = []
      val = int(idx_next[0][0].item())
      res.append(val)

      idx = torch.cat((idx, idx_next), dim=1)

    # Return final sequence after generating max_new_tokens
    return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1,1), dtype=torch.long)
value = decode(m.generate(idx, max_new_tokens=100)[0].tolist())
print(value, len(value))
print(list(value))

torch.Size([32, 65])
tensor(4.6437, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3 101
['\n', 'S', 'r', '?', 'q', 'P', '-', 'Q', 'W', 'k', 't', 'X', 'o', 'L', '&', 'j', 'L', 'D', 'J', 'g', 'O', 'L', 'V', 'z', "'", 'R', 'I', 'o', 'D', 'q', 'H', 'd', 'h', 's', 'V', '&', 'v', 'L', 'L', 'x', 'a', 't', 'j', 's', 'c', 'M', 'p', 'w', 'L', 'E', 'R', 'S', 'P', 'y', 'a', 'o', '.', 'q', 'f', 'z', 's', '$', 'Y', 's', '$', 'z', 'F', '-', 'w', ',', ';', 'e', 'E', 'k', 'z', 'x', 'j', 'g', 'C', 'K', 'F', 'C', 'h', 's', '!', 'i', 'W', 'W', '.', 'O', 'b', 'z', 'D', 'n', 'x', 'A', ' ', 'M', 's', '$', '3']


In [ ]:
# We will keep the generate func like this for now, because rn silly
# and we only using the last token to predict next one

In [78]:
# Train the model
# Create a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [84]:
batch_size = 32
for steps in range(10000):
  xb, yb = get_batch("train")

  logits, loss = m(xb, yb)
  # reset tensors (none so pytorch doesnt have to allocate memory for zero tensors)
  optimizer.zero_grad(set_to_none=True)
  # find out which weights are contributing to the loss, and direction to move
  loss.backward()
  # apply this direction to move towards local minimum (loss func)
  optimizer.step()

print(loss.item())

2.4921743869781494


In [86]:
decode(m.generate(idx, max_new_tokens=400)[0].tolist())

"\nFor ous ay whundlyo d yome PULAXimuriusanro; shes dur'd, s at CEn CURDYWey t havee to d.\nBucee n r owis poboungalknajus fo tze yonout eit r thom t, ch ar t g\nI LOfarsmalle thenierd p ourry, be ho.\n\n\nOLI'd TII thithoubepred dar toris, thaums, athacknthene, traMe ars ame nen; frs thengl courit,\nAn.\nMe seionnghthathus. momees rdoude d the sth\n\nII ingis ifre h pomo guk f toare are:\nSanonogrdinotofifur"